## Inverse Kinematics
### Carter Kruse

---

This Jupyter notebook walks through an exploration of inverse kinematics, using `bqplot`. The following provides a description of inverse kinematics, alongside the code that corresponds to the description.

### Imports

The following import statements are required to show the plots of this Jupyter notebook. In this project, we use a library called **[bqplot](https://github.com/bloomberg/bqplot)** developed by Bloomberg, which enables fully interactive plots within the Jupyter notebook. By default, this library is not part of the Python distribution, so we must install it.

In [1]:
%%capture
%pip install numpy matplotlib bqplot scipy
%jupyter nbextension enable --py --sys-prefix bqplot
%jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [2]:
import numpy as np # Building & Manipulating Matrices

# SciPy
import scipy.linalg as la

# Optimization Package
import scipy.optimize as opt

# Plotting Library
import bqplot as bqp

# Graphical User Interface Components
from ipywidgets import interact
from ipywidgets import FloatSlider, VBox

### Forward Kinematics

First, we will investigate the mathematics a chain of segments in two dimensions with joint positions $ \bm{v_i} = \left( x_i, y_i \right) $. As with the assignment, the first joint is rigidly attached to the origin (i.e. $ \bm{p_0} = \left( 0, 0 \right) $) while the other joints and segments are free to move in any way. We will assume that all of the segments have the same length $ l_1 = l_2 = \ldots = 1 $.

Each parameter $ \theta_i \in [0, 2 \pi] $ specifies the counter-clockwise angle that the associated segment from joint $ \bm{p_{i - 1}} $ to joint $ \bm{p_i} $ makes with its predecessor segment (the pair of segments are parallel if $ \theta_i = 0 $). The first segment does not have a predecessor, hence $ \theta_1 $ is measured relative to the $ x $ axis. Note how the complete set of segment angles $ \theta_1, \theta_2, \ldots $ is all the information we need to compute the precise positions of all the joint positions in Euclidean space.

Forward kinematics is defined as the problem of converting a set of segment angles $ \theta_i $ into joint positions $ \bm{p_i} $. Since $ \bm{p_i} $ depends on all of the preceding angles, we can think of each joint position as a function $ \bm{p_i} = \bm{p \left( \theta_1, \ldots, \theta_{i} \right)} $.

#### A Simple 1-Segment Chain

Now, we create a function `chain_simple`, which solves the forward kinematics for a chain with at most one segment. The function takes in an array of angles as a parameter, which can be of length 0 or 1. When no angles are specified, the function returns the position of the first joint $ \left( x_0, y_0 \right) = \left( 0, 0 \right) $ as a 1D `numpy` array. When a single angle is specified, it returns the position $ x_1, y_1 $.

In [3]:
# Solves the forward kinematics for a chain with at most one segment.
def chain_simple(theta):
    # Base Case
    if len(theta) == 0:
        return np.array([0., 0.])
    
    # Advanced Case
    else:
        return np.array([np.cos(theta[0]), np.sin(theta[0])])

# Tests
print('chain_simple:', chain_simple([]))
print('reference:   ', np.array([0., 0.]))
print()
print('chain_simple:', chain_simple([0.]))
print('reference:   ', np.array([1., 0.]))
print()
print('chain_simple:', chain_simple([np.pi / 4]))
print('reference:   ', np.array([0.70710678, 0.70710678]))

chain_simple: [0. 0.]
reference:    [0. 0.]

chain_simple: [1. 0.]
reference:    [1. 0.]

chain_simple: [0.70710678 0.70710678]
reference:    [0.70710678 0.70710678]


#### Visualization Of The Forward Kinematics

**Helper Function**

We provide the function ``fk_demo()`` below to interactively explore the possible chain configurations via forward kinematics. The implementation uses the ``bqplot`` library mentioned above and is fairly technical.

In [4]:
def fk_demo(chain_func, theta, extra = [[],[]]):
    '''
    This function visualizes the configuration of a chain of segments
    and permits interactive changes to its state. It expects two arguments:
    
    ``chain_func``: A function that implements forward kinematics by
    turning a sequence of angles (theta_1, theta_2, ..., theta_n) into
    the position of the last joint of this chain (x_n, y_n).
    
    ``theta``: An array with the initial angles of all joints.
    
    ``extra``: An optional argument which can be used to plot
    additional points that are highlighted in red.
    '''
    
    # Function which repeatedly calls ``chain_func`` to compute all joint positions.
    def chain_all(theta):
        return np.column_stack([chain_func(theta[:i]) for i in range(0, len(theta) + 1)])

    # Determine Size & Initial Configuration
    size = len(theta)
    positions = chain_all(theta)

    # Define the range of the plotting frame.
    scales = {'x': bqp.LinearScale(min = -size - 1, max = size + 1),
              'y': bqp.LinearScale(min = -size - 1, max = size + 1)}

    # Create a scatter plot (for joints), a line plot (for segments), and
    # another scatter plot (to draw extra points specified the ``extra`` argument).
    scat  = bqp.Scatter(scales = scales)
    lines = bqp.Lines(scales = scales)
    scat2 = bqp.Scatter(scales = scales, colors = ['red'])

    # Create a figure that combines the three plots.
    figure = bqp.Figure(marks = [scat, scat2, lines])
    figure.layout.height = '500px'
    figure.layout.width = '500px'

    # Initialize the plots with the initial data.
    scat.x, scat.y = positions
    lines.x, lines.y = positions
    scat2.x, scat2.y = extra
    
    sliders = []
    
    # Cycling through the angles theta_i.
    for i in range(len(theta)):
        # Create a graphical slider.
        slider = FloatSlider(min = 0, max = 2 * np.pi, value = theta[i], step = 1e-3)
        
        # Define a callback function that will be triggered when the slider is moved.
        def callback(value, i = i):
            theta[i] = value['new']
            positions = chain_all(theta)
            scat.x, scat.y = positions
            lines.x, lines.y = positions

        # "Attach" the callback function to the slider.
        slider.observe(callback, 'value')
        sliders.append(slider)

    # Combine the plots and sliders in a vertical arrangement.
    return VBox([*sliders, figure])

Now, we ensure that the implementation of `chain_simple` satisfies all the specifications by invoking the `fk_demo()` function with arguments `chain_simple` and ``[0.]`` (the initial parameters of a flat chain). You should be able to drag a slider from 0 to $ 2 \pi $ and see a visual representation of a 1-segment chain turning counter-clockwise.

In [5]:
# Visualization
fk_demo(chain_simple, [0.])

#### Longer Chains

Now, we create a function `chain`, which solves the forward kinematics for an arbitrarily long sequence of segments. The function takes an arbitrary-length array of angles as a parameter. When no angles are specified, the function returns the position $ \left( x_0, y_0 \right) $ as before. When $ i $ angles are specified, it (only) returns  the joint position $ \left( x_{i}, y_{i} \right) $. In this case, unlike the assignment, we use recursion, which allows for a particularly simple implementation.

In [6]:
# Solves the forward kinematics for a chain with arbitarily many segments.
def chain(theta):
    # Base Case (Zero - 0 OR One - 1)
    if len(theta) <= 1:
        # Return the simple result (x, y) according to the angle.
        return chain_simple(theta)
    
    # Recursive Case
    else:
        # Create a copy of the array, as it is dynamically modified.
        theta_copy = np.copy(theta)
        
        # Update the angle.
        theta_copy[1] += theta_copy[0]
        
        # Return the simple result (x, y), along with the recursion.
        return chain_simple([theta_copy[0]]) + chain(theta_copy[1:])

# Tests
print('chain:     ', chain([0.1, 0.2, 0.3, 0.4]))
print('reference: ', np.array([3.31597858, 1.80146708]))
print()
print('chain:     ', chain([np.pi, np.pi, np.pi, np.pi]))
print('reference: ', np.array([0., 0.]))

chain:      [3.31597858 1.80146708]
reference:  [3.31597858 1.80146708]

chain:      [ 0.0000000e+00 -2.4492936e-16]
reference:  [0. 0.]


#### Attempting To Reach A Certain Position

Now, let us find a configuration of angles that brings the endpoint of the chain as close as possible to the location `[-2, 3]`. You should see a chain with five segments and five corresponding sliders, as well as an additional point highlighted in red. 

In [7]:
# Demo
fk_demo(chain, [1.00, 0.58, 0.69, 0.40, 0.53], [[-2], [3]])

### Inverse Kinematics

Problems similar to the last one are tedious to solve by hand: all of the parameters are interdependent and must be adjusted in a coordinated manner. So-called *inverse kinematics* techniques apply numerical root finding to determine solutions to this problem in an automated way. Now, we will use inverse kinematics to automatically determine $ \theta_1, \ldots, \theta_n $ such that

$$
\bm{p \left( \theta_1, \ldots, \theta_n \right)} = \bm{p_{\mathrm{target}}}
$$

for a given value $ \bm{p_{\mathrm{target}}} \in \mathbb{R}^2 $. This means that for a given endpoint of the chain, the chain will automatically reconfigure itself to match.

Numerical root finding techniques require the ability to evaluate the Jacobian of $ \bm{p} $, i.e. all the partial derivatives $ \frac{ \partial \bm{p \left( \theta_1, \ldots, \theta_n \right)}}{\partial \theta_j} $. The partial derivatives encode how a small perturbation of each of the angles $ \theta_j $ leads to a corresponding change in $ \bm{p \left( \theta_1, \ldots, \theta_n \right)} $.

#### A Simple 1-Segment Chain

As before, we will first look at a 1-segment chain and then derive a solution for the general problem. We implement a function `dchain_simple(theta)` which takes an array with one entry, and which computes the function $ \frac{\partial \bm{p \left( \theta_1 \right)}}{\partial \theta_1} $. The return value is a two-dimensional array with one column and two rows containing the partial derivatives of the coordinate values $ x_1 $ and $ y_1 $.

In this case, we use an analytic method, rather than approximating the derivatives via finite differences.

In [8]:
# Calculates the Jacobian (inverse kinematics) for a chain with at most one segment.
def dchain_simple(theta):
    # Base Case
    if len(theta) == 0:
        return np.array([[0.], [1.]])
    
    # Advanced Case
    else:
        # Jacobian Values
        a = -np.sin(theta[0])
        b = np.cos(theta[0])
        
        # Remove the negative size before a zero (0).
        if a == 0: a = 0
        
        return np.array([[a], [b]])

# Tests
print('dchain_simple: \n', dchain_simple([0.]))
print('reference:     \n', np.array([[0.], [1.]]))
print()
print('dchain_simple: \n', dchain_simple([np.pi / 4]))
print('reference:     \n', np.array([[-0.70710678], [0.70710678]]))

dchain_simple: 
 [[0.]
 [1.]]
reference:     
 [[0.]
 [1.]]

dchain_simple: 
 [[-0.70710678]
 [ 0.70710678]]
reference:     
 [[-0.70710678]
 [ 0.70710678]]


#### Implementing The Full Jacobian Function

Now that we finished the version for a single segment, we will now turn to the full Jacobian $ \nabla \bm{p \left( \theta_1, \ldots, \theta_n \right)} $, which is a $ 2 \times n $ matrix containing the partial derivatives with respect to all angles. Here, we use a vector version of the [product](https://en.wikipedia.org/wiki/Product_rule) or [chain rule](https://en.wikipedia.org/wiki/Chain_rule) in the implementation. Specifically, note that

$$
\frac{\partial}{\partial t} \left[ \bm{A \left( t \right)} \bm{x \left( t \right)} \right] = \bm{A' \left( t \right)} \bm{x \left( t \right)} + \bm{A \left( t \right)} \bm{x' \left( t \right)}
$$

where $ \bm{A \left( t \right)} $ and $ \bm{x \left( t \right)} $ are a matrix and a vector depending on a parameter $ t $, respectively.

Thus, we implement a function `dchain(theta)` which accepts a 1D array of angles with length $ \ge 1 $ and computes the Jacobian $ \nabla \bm{p \left( \theta_1, \ldots, \theta_n \right)} $, a $ 2 \times n $ matrix.

In [9]:
# Calculates the Jacobian (inverse kinematics) for a chain with arbitrarily many segments.
def dchain(theta):
    # Create a copy of the array, as it is dynamically modified.
    theta_copy = np.copy(theta)
    
    # Create an array, which holds intermediate values.
    arr = []
    
    # Cycle through the array, up to (n - 1) elements.
    for i in range(len(theta) - 1):
        # Appends the simple result (x, y) according to the angle.
        arr.append(dchain_simple(theta_copy))
        
        # Update the angle.
        theta_copy[1] += theta_copy[0]
        
        # Update the array start.
        theta_copy = theta_copy[1:]
        
    # Appends the simple result (x, y) according to the angle (after cycle).
    arr.append(dchain_simple(theta_copy))
    
    # Create the final result array.
    result = []
    
    # Cycle through the array.
    for i in range(len(theta)):
        # Add the sum of elements, starting at index i.
        result.append(sum(arr[i:]))
    
    # Return the result, reshaped and ordered (as appropriate).
    return np.array(result).flatten().reshape((2, len(theta)), order = 'F')
   
# Tests
print('dchain: \n', dchain([0., 0., 0., 0.]))
print('reference: \n', np.array([[0., 0., 0., 0.], [4., 3., 2., 1.]]))
print()
print('dchain: \n', dchain([0.1, 0.2, 0.3]))
print('reference: \n', np.array([[-0.9599961, -0.86016268, -0.56464247], [2.77567627, 1.7806721, 0.82533561]]))

dchain: 
 [[0. 0. 0. 0.]
 [4. 3. 2. 1.]]
reference: 
 [[0. 0. 0. 0.]
 [4. 3. 2. 1.]]

dchain: 
 [[-0.9599961  -0.86016268 -0.56464247]
 [ 2.77567627  1.7806721   0.82533561]]
reference: 
 [[-0.9599961  -0.86016268 -0.56464247]
 [ 2.77567627  1.7806721   0.82533561]]


#### Solving The Inverse Kinematics Problem Using Newton's Method

Newton's method is one of the most widely used methods for finding solutions to systems of non-linear equations. It converges at a remarkable speed when started sufficiently close to a root, though there is generally no strict guarantee of convergence.

Given a function $  \bm{f \left( \bm{x} \right)} $, Newton's method tries to find a solution to the equation $ \bm{f} = \mathbf{0} $ using steps of the form

$$
\bm{x_{i + 1}} = \bm{x_i} - \left( \nabla \bm{f \left( \bm{x_{i}} \right)} \right)^{-1} \bm{f \left( \bm{x_{i}} \right)}
$$

---

In the context of inverse kinematics, we want to apply Newton's method to solve an equation of the form

$$
\bm{p \left( \theta_1, \ldots, \theta_n \right)} = \bm{p_{\mathrm{target}}}
$$

for a given reference position $ \bm{p_{\mathrm{target}}} \in \mathbb{R}^2 $.

---

In other words, the unknowns are the angles $ \theta_1, \ldots, \theta_n $, and the function whose root we seek maps to a two-dimensional domain. It is not immediately obvious how to apply Newton's method, since the Jacobian of the function has the shape $ 2 \times n $ and hence cannot be inverted using standard techniques like LU decomposition.

This is a consequence of the fact that many different configurations can be used to reach the same $ \bm{p_{\mathrm{target}}} $. Fortunately, we can use the *pseudo-inverse*, a generalization of the inverse to non-square matrices. In this specific case, the Jacobian is *wide* (it has more columns than rows), so the pseudo-inverse will find the solution to a linear system which has the smallest $ \| \cdot \|_2 $-norm.

Now we implement a function `newton(theta, target)` that takes a 1-dimensional array of angles as a starting guess as well as a 2D target position (specified as a 1-dimensional array) as input. The implementation performs a fixed `8` iterations of Newton's method to try to solve the equation $ \bm{p \left( \theta_1, \ldots, \theta_n \right)} = \bm{p_{\mathrm{target}}} $ and returns the final set of parameters $ \theta_1, \ldots, \theta_n $ as an 1-dimensional `numpy` array. The function `la.pinv` is used to compute the *pseudo-inverse*.

In [10]:
# Boolean value used for error logging.
logging = False

# Calculates the Jacobian (inverse kinematics) for a chain with arbitrarily many segments.
def newton(theta, target):    
    # Save the length of the chain, for the loop and reshaping.
    chain_len = len(theta)
    
    for i in range(8):
        # Error Logging
        if logging: print('LOOP {}'.format(i))
        if logging: print('THETA \n {}'.format(theta))
        
        # Calculates the difference between the calculated point and the target.
        f_x = chain(theta) - target
        
        if logging: print('F_X \n {}'.format(f_x))
        
        # Calculates the change (matrix multiplication of inverse of Jacobian and function).
        change = np.matmul(la.pinv(dchain(theta)), f_x.reshape((2, 1))) # % (2 * np.pi)
        
        # Error Logging
        if logging: print('CHANGE \n {}'.format(change))
        
        # Updating 'theta' (the list of angles).
        theta = (theta.reshape((chain_len, 1)) - change).flatten()
    
    # Error Logging
    if logging: print()
    
    return theta

# Tests
# Moving a 1-element chain from the default configuration to position (0., 1.)
print('chain(newton): ', chain(newton(np.array([0.]), np.array([0., 1.]))))
print('reference:     ', np.array([0., 1.]))
print()

# Moving a 2-element chain from the default configuration to position (0.5, 0.5)
print('chain(newton): ', chain(newton(np.array([0., 0.]), np.array([0.5, 0.5]))))
print('reference:     ', np.array([0.5, 0.5]))

chain(newton):  [6.123234e-17 1.000000e+00]
reference:      [0. 1.]

chain(newton):  [0.5 0.5]
reference:      [0.5 0.5]
